In [18]:
import mysql.connector
from dbconfig import db_config

import pandas as pd

In [5]:
mydb = mysql.connector.connect(
        host = db_config["host"],
        user = db_config["user"],
        passwd = db_config["password"],
        database = db_config["database"]
    )
mycursor = mydb.cursor()

In [16]:
mycursor.execute("SELECT * FROM Stocks")
aapl = mycursor.fetchall()
sequence = mycursor.column_names

In [34]:
df = pd.DataFrame(aapl, columns = sequence).set_index('date').drop(['symbol', 'high', 'low', 'open'], axis = 1)
df

,close,volume
date,,
2015-06-01,133.31,33166322
2015-06-02,131.26,35078387
2015-06-03,135.94,32034911
2015-06-04,131.78,38463484
2015-06-05,131.58,37314004
...,...,...
2020-05-22,330.48,20958094
2020-05-26,329.79,32870796
2020-05-27,329.09,28621405


In [38]:
1259/5

251.8